<a href="https://colab.research.google.com/github/mathewpolonsky/Request-Topic-Classification/blob/main/training_rubert_base_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Среда

The following cell fixes the background color of tqdm background color in VSCode Jupyter Notebooks:

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
</style>

Google Colab

In [ ]:
# dataset
!gdown 1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK

# clear dataset
!gdown 1vzYpVcquBvzX5Ige3klpaACQFbjEP4Ak

# id2label and label2id
!gdown 1yBppNyzNCS5tinBvlTIyuMbBDmQhmKBF

!gdown 1GvsfK3vZIBbYViI-KFPCsW-mFw4RUjqK

# contractor dataset
!gdown 1j528C3llhpycw5mqSlUO8hATZR1hzoza

# contractor id2label and label2id
!gdown 1-0o2i16oGXe8gtiV_HnZXzLGiJtfBv9T
!gdown 1FwH6xxW0KXStqkn8nfaeYnsYlars8P_M

# topic2big_topic
!gdown 1EJfpWAHRlgGE9DdPQNYu69hoUDmbahT0

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
# !pip install -U sentence-transformers
!pip install evaluate
!pip install transformers[torch]
!pip install demoji

---

### Импорт библиотек

In [1]:
import re
import json

import numpy as np
import pandas as pd

import demoji

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import DataLoader, Dataset
import evaluate

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast, pipeline

from tqdm.auto import tqdm

### Обработка датасета

In [6]:
data = pd.read_csv('train_dataset_train.csv', sep=';')
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу


Убираем символ `'` в начале всех текстов инцидента

In [ ]:
all([i[0] == "'" for i in data['Текст инцидента'].values])

True

In [ ]:
data['Текст инцидента'] = data['Текст инцидента'].str.strip("'")
data['Текст инцидента'].head()

0    Добрый день. Сегодня, 20.08.22, моя мать шла п...
1    Пермь г, +79194692145. В Перми с ноября 2021 г...
2    Добрый день ! Скажите пожалуйста если подовала...
3    Каждая из них не о чем. Люди на остановках хот...
4    В Березниках у сына привитого откоронавируса з...
Name: Текст инцидента, dtype: object

Убираем тег `<br>` в начале всех текстов инцидента

In [ ]:
data['Текст инцидента'].str.contains('<br>').sum()

3329

In [ ]:
data['Текст инцидента'] = data['Текст инцидента'].str.replace('<br>', '\n')

Убираем ссылку на пользователя, которому адресован комментарий

In [ ]:
data[data['Текст инцидента'].str.startswith("[")]['Текст инцидента']

22       [club185980418|Центр социальных выплат Пермско...
25       [club57433185|Пермь Первая], обратите внимание...
38       [id269738613|Дмитрий], в Краснокамске тоже ест...
59       [club80949945|Администрация города Лысьвы], ко...
64       [club201789187|ЦУР Пермского края] , здравству...
                               ...                        
23100    [club57433185|Пермь Первая], проблема с люком ...
23105    [id586879673|Жанна], Я дважды уже столкнулась ...
23113    [id153709709|Нина], ходить не возможно даже та...
23120    [club171874188|МАУ "СШ армейского рукопашного ...
23122    [club173907682|Березники официальные], а если ...
Name: Текст инцидента, Length: 1470, dtype: object

In [ ]:
def remove_recipient(text):
    pattern = r"^\[[^\]]+\]"

    text_wo_recipient = re.sub(pattern, "", text).strip(', ')
    return text_wo_recipient

In [ ]:
text = data[data['Текст инцидента'].str.startswith("[")]['Текст инцидента'].iloc[1]
print(text)

remove_recipient(text)

[club57433185|Пермь Первая], обратите внимание на организацию работы на ГЭС и на незаконченный ремонт дороги через переезд на ул. Писарева!


'обратите внимание на организацию работы на ГЭС и на незаконченный ремонт дороги через переезд на ул. Писарева!'

In [ ]:
data['Текст инцидента'] = data['Текст инцидента'].apply(remove_recipient)

Если в тексте инцидента менее чем 4 слова, убираем такой текст

In [ ]:
(data['Текст инцидента'].str.split().apply(len) < 4).sum()

638

In [ ]:
data = data[data['Текст инцидента'].str.split().apply(len) >= 4]
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла п...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, +79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день ! Скажите пожалуйста если подовала...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса з...,Технические проблемы с записью на прием к врачу


In [ ]:
# После того, как убрали ссылки, первые предложения могут начинаться
# с маленькой буквы
data['Текст инцидента'] = data['Текст инцидента'].apply(lambda text: text[0].upper() + text[1:])

Убираем эмоджи

In [ ]:
data["Текст инцидента"] = data["Текст инцидента"].apply(lambda x: demoji.replace(x, ""))

### Spellchecker

In [ ]:
class SpellDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return 'Spell correct: ' + self.original_list[i]

spell_dataset = SpellDataset(data['Текст инцидента'].values)

In [ ]:
spell_dataset[0]

'Spell correct: Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.\nУважаемая Администрация, сделайте с этим что-нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо!'

In [ ]:
spell_pipeline = pipeline(model='UrukHan/t5-russian-spell',
                          task='text2text-generation', batch_size=64, device='cuda')

In [ ]:
spells = []

for out in tqdm(spell_pipeline(spell_dataset), total=len(spell_dataset)):
    spells.append(out)

In [ ]:
spells

['«Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Спасибо, шла не одна. Уважаемая Администрация, сделайте с этим что-нибудь. Да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо! Спасибо! Спасибо! Спасибо!! Спасибо!! Спасибо!!',
 'Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2 группы? Проезд в общественном транспорте не представляется.',
 'Здравствуйте! Скажите, пожалуйста, если подала на пособие с 3 до 7 декабря, когда можно повторно подать? . . . . Когда можно повторно подать? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Вроде за 30 дней можно.',
 'А люди на остановках хотят укрыться от непогоды или слишком погоды. Присесть, поставить сумку. Лавочки на полторы попы? Отсутствие или намек на 

In [ ]:
def check_spelling(input_text):
    try:
        encoded = spell_tokenizer(
            task_prefix + input_text,
            padding="longest",
            max_length=max_input,
            truncation=True,
            return_tensors="pt",
        ).to('cuda')

        predicts = spell_model.generate(**encoded)
        correct_text = spell_tokenizer.batch_decode(predicts, skip_special_tokens=True)[0]

        # Убираем лишние символы в начале предложения, если модель их добавила
        # correct_text = correct_text[correct_text.index(input_text[0]):]
        correct_text = correct_text.lstrip('.,[]«»')

        # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
        correct_text = re.sub(r'([^\w\s])\1+', r'\1', correct_text)
    except:
        print(correct_text)

    return correct_text

In [ ]:
# n = random.randint(0, data.shape[0])
n = 22
# print(n)

print(remove_recipient(data['Текст инцидента'][n]))
check_spelling(data['Текст инцидента'][n])

А какие выплаты для малоимущей(малообеспеченной) неполной семьи есть в вашем центре,весь доход семьи 10000 т.р. с небольшим


'А какие выплаты для малоимущей (малообеспеченной) неполной семьи есть в вашем центре? Весь доход семьи 10000 т. р. с небольшим.'

In [ ]:
data['Текст инцидента'] = data['Текст инцидента'].apply(check_spelling)

---

### Классификация

jfbthtrht

In [2]:
data = pd.read_csv('data_corrected_spell_ner_full_text.csv')
data

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners,full_text_wo_contractor
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина,"Добрый день. Сегодня, 20 августа, моя мать шла..."
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми","Пермь, г. , +791692145. В Перми с ноября 2021 ..."
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,NaN,"Добрый день! Скажите, пожалуйста, если подала ..."
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,NaN,Каждая из них не о чем. Люди на остановках хот...
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках,"В Березниках у сына, привитого от коронавируса..."
...,...,...,...,...,...,...
22485,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,NaN,"А если ещё не погасили ипотеку, но площадь бол..."
22486,Губахинский городской округ,ЖКХ,Город Гремячинск — ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,LOC: Гремячинск,Город Гремячинск — ситуация с теплом на улице ...
22487,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...",Технические проблемы с записью на прием к врачу,NaN,"Здравствуйте, у меня ребёнку 2 месяца. Тест на..."
22488,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов?! Воо...,Благоустройство придомовых территорий,LOC: Оборина,А что творится с благоустройством дворов?! Воо...


In [3]:
def remove_extra_symbols(text):
    # Убираем лишние символы в начале предложения, если модель их добавила
    # correct_text = correct_text[correct_text.index(input_text[0]):]
    text = text.lstrip('.,[]«»')

    # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
    text = re.sub(r'([^\w\s])\1+', r'\1', text)

    return text

data['Текст инцидента'] = data['Текст инцидента'].apply(remove_extra_symbols)
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners,full_text_wo_contractor
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина,"Добрый день. Сегодня, 20 августа, моя мать шла..."
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми","Пермь, г. , +791692145. В Перми с ноября 2021 ..."
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,NaN,"Добрый день! Скажите, пожалуйста, если подала ..."
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,NaN,Каждая из них не о чем. Люди на остановках хот...
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках,"В Березниках у сына, привитого от коронавируса..."


In [4]:
def text_w_ners(row):
    full_text = f"{row['Текст инцидента']};"

    if row['Ners']:
        full_text += f"\n{row['Ners']}"

    return full_text

data['text_w_ners'] = data[['Текст инцидента', 'Ners']].apply(text_w_ners, axis=1)
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners,full_text_wo_contractor,text_w_ners
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина,"Добрый день. Сегодня, 20 августа, моя мать шла...","Добрый день. Сегодня, 20 августа, моя мать шла..."
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми","Пермь, г. , +791692145. В Перми с ноября 2021 ...","Пермь, г. , +791692145. В Перми с ноября 2021 ..."
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,NaN,"Добрый день! Скажите, пожалуйста, если подала ...","Добрый день! Скажите, пожалуйста, если подала ..."
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,NaN,Каждая из них не о чем. Люди на остановках хот...,Каждая из них не о чем. Люди на остановках хот...
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках,"В Березниках у сына, привитого от коронавируса...","В Березниках у сына, привитого от коронавируса..."


In [5]:
print(data['text_w_ners'].loc[0])

Добрый день. Сегодня, 20 августа, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Спасибо, уважаемая администрация, сделайте с этим что-нибудь. Да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.;
LOC: Ленина


In [5]:
def get_id_and_labels():
    id2label_path = 'id2label.json'
    label2id_path = 'label2id.json'

    with open(id2label_path, 'r', encoding='UTF-8') as file:
        id2label = json.load(file)

    id2label = {int(key):value for key,value in id2label.items()}

    with open(label2id_path, 'r', encoding='UTF-8') as file:
        label2id = json.load(file)

    return id2label, label2id

id2label, label2id = get_id_and_labels()

#### Обучение

In [ ]:
data['Тема'].unique().shape

(195,)

In [6]:
data['label'] = [label2id[topic] for topic in data['Тема']]
data

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners,full_text_wo_contractor,text_w_ners,label
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина,"Добрый день. Сегодня, 20 августа, моя мать шла...","Добрый день. Сегодня, 20 августа, моя мать шла...",0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми","Пермь, г. , +791692145. В Перми с ноября 2021 ...","Пермь, г. , +791692145. В Перми с ноября 2021 ...",1
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,NaN,"Добрый день! Скажите, пожалуйста, если подала ...","Добрый день! Скажите, пожалуйста, если подала ...",2
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,NaN,Каждая из них не о чем. Люди на остановках хот...,Каждая из них не о чем. Люди на остановках хот...,3
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках,"В Березниках у сына, привитого от коронавируса...","В Березниках у сына, привитого от коронавируса...",4
...,...,...,...,...,...,...,...,...
22485,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,NaN,"А если ещё не погасили ипотеку, но площадь бол...","А если ещё не погасили ипотеку, но площадь бол...",125
22486,Губахинский городской округ,ЖКХ,Город Гремячинск — ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,LOC: Гремячинск,Город Гремячинск — ситуация с теплом на улице ...,Город Гремячинск — ситуация с теплом на улице ...,44
22487,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...",Технические проблемы с записью на прием к врачу,NaN,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...","Здравствуйте, у меня ребёнку 2 месяца. Тест на...",4
22488,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов?! Воо...,Благоустройство придомовых территорий,LOC: Оборина,А что творится с благоустройством дворов?! Воо...,А что творится с благоустройством дворов?! Воо...,122


In [7]:
# checkpoint = "xlm-roberta-base"
# classification_checkpoint = "cointegrated/rubert-tiny2"
classification_checkpoint = "ai-forever/ruBert-base"

tokenizer = AutoTokenizer.from_pretrained(classification_checkpoint)

classification_model = AutoModelForSequenceClassification.from_pretrained(
    classification_checkpoint, num_labels = data['Тема'].unique().shape[0],
    id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
class_weights = compute_class_weight(None, classes=np.array(list(label2id.keys())), y=data["Тема"])
class_weights = torch.tensor(class_weights, device=classification_model.device).to(torch.float).to("cuda")

In [9]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        outputs = classification_model(**inputs)
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [10]:
train_data, val_data = train_test_split(
    data[['text_w_ners', 'label']], random_state=42, test_size=.1
)
val_data.head()

,text_w_ners,label
13586,В Мотовилихе тоже не везде порядок с остановка...,3
19217,В посёлке станции «Уральская» не разъехаться. ...,9
7201,"Скажите, пожалуйста! Где можно пройти окулиста...",7
16776,Я сегодня целый день обзванивала всех: поликли...,4
17831,"Мусор в Сарашах по середине улицы, позор и сты...",25


In [11]:
class TextDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences = data_df["text_w_ners"].values
        self.labels = data_df['label'].values

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, i):
        sentence, label = self.sentences[i], self.labels[i]

        tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens['labels'] = label

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        return tokens


train_dataset = TextDataset(train_data, tokenizer)
val_dataset = TextDataset(val_data, tokenizer)

# train_dataset[0]

In [12]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    out = {}

    out.update(accuracy.compute(predictions=predictions, references=labels))
    out.update(f1_metric.compute(predictions=predictions, references=labels,
                                 average='weighted'))

    return out

In [14]:
training_args = TrainingArguments(
    output_dir="promobot/models/rubert_base",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [15]:
trainer = CustomTrainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.399300,2.277277,0.481547,0.404066
2,1.808700,2.004377,0.514896,0.448527
3,1.530300,1.949246,0.539351,0.484906


TrainOutput(global_step=7593, training_loss=2.1271103429775473, metrics={'train_runtime': 5922.5551, 'train_samples_per_second': 10.253, 'train_steps_per_second': 1.282, 'total_flos': 1.6004578478533632e+16, 'train_loss': 2.1271103429775473, 'epoch': 3.0})

In [29]:
classification_model.save_pretrained("ruBert-base-topic-ner/")

!zip -r ruBert-base-topic-ner.zip ruBert-base-topic-ner

!cp ruBert-base-topic-ner.zip /content/drive/MyDrive/

  adding: ruBert-base-topic-ner/ (stored 0%)
  adding: ruBert-base-topic-ner/config.json (deflated 88%)
  adding: ruBert-base-topic-ner/model.safetensors (deflated 7%)


In [20]:
class_weights = compute_class_weight('balanced', classes=np.array(list(label2id.keys())), y=data["Тема"])
class_weights = torch.tensor(class_weights, device=classification_model.device).to(torch.float).to("cuda")

In [21]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        outputs = classification_model(**inputs)
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [23]:
training_args = TrainingArguments(
    output_dir="promobot/models/rubert_base",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [24]:
trainer = CustomTrainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.259300,2.834150,0.517563,0.499881


TrainOutput(global_step=2531, training_loss=2.4170773527642004, metrics={'train_runtime': 1972.6343, 'train_samples_per_second': 10.261, 'train_steps_per_second': 1.283, 'total_flos': 5334859492844544.0, 'train_loss': 2.4170773527642004, 'epoch': 1.0})

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.835800,2.673682,0.501556,0.493538


TrainOutput(global_step=2531, training_loss=1.927749630774683, metrics={'train_runtime': 1973.767, 'train_samples_per_second': 10.255, 'train_steps_per_second': 1.282, 'total_flos': 5334859492844544.0, 'train_loss': 1.927749630774683, 'epoch': 1.0})

#### Тест

In [ ]:
n = np.random.randint(val_data.shape[0])
n

In [ ]:
sentence = str(val_data["Текст инцидента"].iloc[n])

print(sentence, '\n', id2label[val_data["label"].iloc[n]])

tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=512)

tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}


for key in tokens:
    # tokens[key] = tokens[key].to("cuda").unsqueeze(0)
    tokens[key] = tokens[key].unsqueeze(0)

In [ ]:
pred = model(**tokens)
id2label[pred["logits"].argmax().item()]